In [35]:
# coding: utf-8

# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

"""
An implementation of benchmarks 1 and 2 from

    Brette et al. (2007) Journal of Computational Neuroscience 23: 349-398

The IF network is based on the CUBA and COBA models of Vogels & Abbott
(J. Neurosci, 2005).  The model consists of a network of excitatory and
inhibitory neurons, connected via current-based "exponential"
synapses (instantaneous rise, exponential decay).

Andrew Davison, UNIC, CNRS
August 2006
"""
import socket
import spynnaker8 as p
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.utility import Timer
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt
import numpy as np

simulator_name = 'spiNNaker'
benchmark = 'CUBA'

# exec("from pyNN.%s import *" % simulator_name)

timer = Timer()

# === Define parameters ===
threads = 1
rngseed = 98766987
parallel_safe = True

n = 1500          # number of cells
r_ei = 4.0        # number of excitatory cells:number of inhibitory cells
pconn = 0.02      # connection probability
stim_dur = 1000.    # (ms) duration of random stimulation
rate = 100.       # (Hz) frequency of the random stimulation

dt = 1.0          # (ms) simulation timestep
tstop = 1000      # (ms) simulaton duration
delay = 2

# Cell parameters
area = 20000.     # (µm²)
tau_m = 20.       # (ms)
cm = 1.           # (µF/cm²)
g_leak = 5e-5     # (S/cm²)

E_leak = -60.  # (mV)
v_thresh = -50.   # (mV)
v_reset = -60.    # (mV)
t_refrac = 5.     # (ms) (clamped at v_reset)
v_mean = -60.     # (mV) mean membrane potential, for calculating CUBA weights
tau_exc = 5.      # (ms)
tau_inh = 10.     # (ms)

# Synapse parameters

Gexc = 4.     # (nS)
Ginh = 51.    # (nS)
Erev_exc = 0.     # (mV)
Erev_inh = -80.   # (mV)

# === Calculate derived parameters ===
area = area * 1e-8                     # convert to cm²
cm = cm * area * 1000                  # convert to nF
Rm = 1e-6 / (g_leak * area)            # membrane resistance in MΩ
assert tau_m == cm * Rm                # just to check

n_exc = int(round((n * r_ei / (1 + r_ei))))  # number of excitatory cells
n_inh = n - n_exc                            # number of inhibitory cells

print("{} {}".format(n_exc, n_inh))


celltype = p.IF_cond_exp
w_exc = Gexc * 1e-3              # We convert conductances to uS
w_inh = Ginh * 1e-3
print("{} {}".format(w_exc, w_inh))

# === Build the network ===

extra = {'threads': threads,
         'filename': "va_%s.xml" % benchmark,
         'label': 'VA'}
if simulator_name == "neuroml":
    extra["file"] = "VAbenchmarks.xml"

node_id = p.setup(
    timestep=dt, min_delay=delay, max_delay=delay,
    db_name='va_benchmark.sqlite', **extra)

cell_params = {'tau_m': tau_m,
               'tau_syn_E': tau_exc,
               'tau_syn_I': tau_inh,
               'v_rest': E_leak,
               'v_reset': v_reset,
               'v_thresh': v_thresh,
               'cm': cm,
               'tau_refrac': t_refrac,
               'i_offset': 0
               }

print(cell_params)

if (benchmark == "COBA"):
    cell_params['e_rev_E'] = Erev_exc
    cell_params['e_rev_I'] = Erev_inh

timer.start()

print("%s Creating cell populations..." % node_id)
exc_cells = p.Population(
    n_exc, celltype(**cell_params), label="Excitatory_Cells")
inh_cells = p.Population(
    n_inh, celltype(**cell_params), label="Inhibitory_Cells")

print("%s Initialising membrane potential to random values..." % node_id)
rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
uniformDistr = RandomDistribution('uniform', [v_reset, v_thresh], rng=rng)
exc_cells.initialize(v=uniformDistr)
inh_cells.initialize(v=uniformDistr)


# manual function 
#
# define the list of spike sources, e.g, with a gaussian input 

n_cells['thalamus' ] = n_cells['exc']
spike_times = [[]]*n_cells['thalamus' ] #list of spike lists, where one spike list is related to one spike source

# randomly select the id source
random_sources_idx = [np.random.randint(n_cells['thalamus' ]*0.25, n_cells['thalamus' ]*0.75) for i in range(n_cells['thalamus' ])]

# randomly assign for each id source a sequence of spike times, i.e., when the cell spikes
for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = np.sort([np.random.normal(loc=500, scale=10) for n in range(5)])

spike_times
%matplotlib
a = plt.eventplot(spike_times)
plt.xlabel('[ms]')
plt.ylabel('Thalamic cells')
plt.title('Raster plot of input spike sources')
plt.xlim(0, 1000)

#spike_times[400:600]
# add plot spikes/sec




 thalamic stimulus
ext_stim = p.Population(
    20, p.SpikeSourcePoisson(rate=rate, duration=stim_dur),
    label="expoisson")
rconn = 0.01
ext_conn = p.FixedProbabilityConnector(rconn)

connections = {}
connections['ext2e'] = p.Projection(
    ext_stim, exc_cells, ext_conn, receptor_type='excitatory',
    synapse_type=p.StaticSynapse(weight=0.1))
connections['ext2i'] = p.Projection(
    ext_stim, inh_cells, ext_conn, receptor_type='excitatory',
    synapse_type=p.StaticSynapse(weight=0.1))

min_delay = 1
max_delay = min_delay*144
d0 = 0.04
v = 0.5
distance = []
delay = []
weight_exc = w_exc
weight_inh = w_inh
conn_list_exc = []
conn_list_inh = []

np.random.seed(7)
for i in range(1500):
    for j in range(1500):
         prob = np.random.uniform(0,1)
         if prob <= np.random.normal(loc=0.0225, scale=0.000):
            distance_ = np.linalg.norm(i-j)
            delay_ = min_delay + distance_ * d0 / v
            #delay_ = np.maximum(delay_, min_delay)
            #delay_ = np.minimum(delay_, max_delay-min_delay) # 
            conn_list_exc += [[i, j, weight_exc, delay_]]
            conn_list_inh += [[i, j, weight_inh, delay_]]

connections['e2e'] = p.Projection(exc_cells, 
                            exc_cells,
                            connector = p.FromListConnector(conn_list_exc),
                            synapse_type = None,
                            receptor_type = 'excitatory',
                            label = 'e2e'
                            )

connections['e2i'] = p.Projection(exc_cells, 
                             inh_cells,
                            connector = p.FromListConnector(conn_list_exc),
                            synapse_type = None,
                            receptor_type = 'excitatory',
                            label = 'e2i'
                            )

connections['i2e'] = p.Projection(inh_cells, 
                             exc_cells,
                            connector = p.FromListConnector(conn_list_inh),
                            synapse_type = None,
                            receptor_type = 'inhibitory',
                            label = 'i2e'
                            )

connections['i2i'] = p.Projection(inh_cells, 
                             inh_cells,
                            connector = p.FromListConnector(conn_list_inh),
                            synapse_type = None,
                            receptor_type = 'inhibitory',
                            label = 'i2i'
                            )

# === Setup recording ===
print("%s Setting up recording..." % node_id)
exc_cells.record(["spikes", 'v', 'gsyn_inh', 'gsyn_exc'])
inh_cells.record(["spikes", 'v', 'gsyn_inh', 'gsyn_exc'])
ext_stim.record("spikes")


buildCPUTime = timer.diff()

# === Run simulation ===
print("%d Running simulation..." % node_id)

print("timings: number of neurons: {}".format(n))
print("timings: number of synapses: {}".format(n * n * pconn))

p.run(tstop)

simCPUTime = timer.diff()

# === Print results to file ===

exc_obs = exc_cells.get_data(['spikes', 'v', 'gsyn_inh', 'gsyn_exc'])
inh_obs = inh_cells.get_data(['spikes', 'v', 'gsyn_inh', 'gsyn_exc'])
stim_obs = ext_stim.get_data(['spikes'])


writeCPUTime = timer.diff()
np = 1
if node_id == 0:
    print("\n--- Vogels-Abbott Network Simulation ---")
    print("Nodes                  : %d" % np)
    print("Simulation type        : %s" % benchmark)
    print("Number of Neurons      : %d" % n)
    print("Number of Synapses     : %s" % connections)
    print("Excitatory conductance : %g nS" % Gexc)
    print("Inhibitory conductance : %g nS" % Ginh)
    print("Build time             : %g s" % buildCPUTime)
    print("Simulation time        : %g s" % simCPUTime)
    print("Writing time           : %g s" % writeCPUTime)


# === Finished with simulator ===

p.end()



2022-01-05 18:56:09 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2022-01-05 18:56:09 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-01-05 18:56:09 INFO: Setting time scale factor to 1.
2022-01-05 18:56:09 INFO: Setting machine time step to 1000 micro-seconds.
2022-01-05 18:56:09 WARNING: Extra params {'db_name': 'va_benchmark.sqlite', 'threads': 1, 'filename': 'va_CUBA.xml', 'label': 'VA'} have been applied to the setup command which we do not consider


1200 300
0.004 0.051000000000000004
['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']
{'tau_m': 20.0, 'tau_syn_E': 5.0, 'tau_syn_I': 10.0, 'v_rest': -60.0, 'v_reset': -60.0, 'v_thresh': -50.0, 'cm': 0.2, 'tau_refrac': 5.0, 'i_offset': 0}
0 Creating cell populations...
0 Initialising membrane potential to random values...


2022-01-05 18:56:19 WARNING: The end user entered a max delay for which the projection breaks
2022-01-05 18:56:19 WARNING: The end user entered a max delay for which the projection breaks
2022-01-05 18:56:19 WARNING: The end user entered a max delay for which the projection breaks
2022-01-05 18:56:19 WARNING: The end user entered a max delay for which the projection breaks
2022-01-05 18:56:19 INFO: Simulating for 1000 1.0ms timesteps using a hardware timestep of 1000us
2022-01-05 18:56:19 INFO: Starting execution process


0 Setting up recording...
0 Running simulation...
timings: number of neurons: 1500
timings: number of synapses: 45000.0


2022-01-05 18:56:23 INFO: Time 0:00:04.198742 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2022-01-05 18:56:32 INFO: Time 0:00:08.875658 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2022-01-05 18:56:37 INFO: Time 0:00:04.922650 taken by PartitionAndPlacePartitioner
Created spalloc job 6233476
2022-01-05 18:56:37 INFO: Created spalloc job 6233476
Waiting for board power commands to complete.
2022-01-05 18:56:37 INFO: Waiting for board power commands to complete.
2022-01-05 18:56:42 INFO: Time 0:00:05.051561 taken by SpallocAllocator
2022-01-05 18:56:42 INFO: Creating transceiver for 10.11.193.17
2022-01-05 18:56:42 INFO: Working out if machine is booted
2022-01-05 18:56:46 INFO: Attempting to


--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Simulation type        : CUBA
Number of Neurons      : 1500
Number of Synapses     : {'ext2e': projection from pre expoisson to post Excitatory_Cells with connector FixedProbabilityConnector(0.01), 'ext2i': projection from pre expoisson to post Inhibitory_Cells with connector FixedProbabilityConnector(0.01), 'e2e': projection e2e, 'e2i': projection e2i, 'i2e': projection i2e, 'i2i': projection i2i}
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Build time             : 10.3893 s
Simulation time        : 69.4294 s
Writing time           : 0.581548 s


In [36]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].hist(np.asarray(exc_obs.segments[0].analogsignals[0]).ravel(), 100)
p=ax[1].hist(np.asarray(exc_obs.segments[0].analogsignals[1]).ravel(), 100)
p=ax[2].hist(np.asarray(exc_obs.segments[0].analogsignals[2]).ravel(), 100)

p=ax[0].hist(np.asarray(inh_obs.segments[0].analogsignals[0]).ravel(), 100)
p=ax[1].hist(np.asarray(inh_obs.segments[0].analogsignals[1]).ravel(), 100)
p=ax[2].hist(np.asarray(inh_obs.segments[0].analogsignals[2]).ravel(), 100)
ax[2].set_xlim(0,0.05)
ax[1].set_xlim(0,0.3)

import matplotlib.colors as colors
fig, ax = plt.subplots(2,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].matshow(np.asarray(exc_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=1))
p=ax[1].matshow(np.asarray(exc_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[2].matshow(np.asarray(exc_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[3].matshow(np.asarray(inh_obs.segments[0].analogsignals[0]).T,  norm=colors.PowerNorm(gamma=1))
p=ax[4].matshow(np.asarray(inh_obs.segments[0].analogsignals[1]).T,  norm=colors.PowerNorm(gamma=0.25))
p=ax[5].matshow(np.asarray(inh_obs.segments[0].analogsignals[2]).T,  norm=colors.PowerNorm(gamma=0.25))

# study network oscillations

fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].eventplot(stim_obs.segments[0].spiketrains)
p=ax[1].eventplot(exc_obs.segments[0].spiketrains)
p=ax[2].eventplot(inh_obs.segments[0].spiketrains)

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
from scipy.fft import fft, fftfreq
from scipy.stats import zscore as z
# Number of samples in normalized_tone

# case of 1000 ms simulations
sample_rate = len(np.asarray(exc_obs.segments[0].analogsignals[0]))
N = 1 * sample_rate

yf0 = fft(np.asarray(exc_obs.segments[0].analogsignals[0]))
yf1 = fft(np.asarray(exc_obs.segments[0].analogsignals[1]))
yf2 = fft(np.asarray(exc_obs.segments[0].analogsignals[2]))

xf = fftfreq(N, 1 / sample_rate)
%matplotlib
import matplotlib.colors as colors
fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
#a=plt.plot(xf, np.abs(yf0))
#a=plt.plot(xf, np.abs(yf1))
a=ax[0].plot(xf, np.abs(yf0), '+', color='b')
a=ax[1].plot(xf, np.abs(yf1), '+', color='k')
a=ax[2].plot(xf, np.abs(yf2), '+', color='r')
#[ax[i].set_xlim(0,1) for i in [0,1,2]]

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
from scipy.fft import fft, fftfreq
from scipy.stats import zscore as z
# Number of samples in normalized_tone

# case of 1000 ms simulations
sample_rate = len(np.asarray(exc_obs.segments[0].analogsignals[0]))
N = 1 * sample_rate

yf0 = fft(np.asarray(inh_obs.segments[0].analogsignals[0]))
yf1 = fft(np.asarray(inh_obs.segments[0].analogsignals[1]))
yf2 = fft(np.asarray(inh_obs.segments[0].analogsignals[2]))

xf = fftfreq(N, 1 / sample_rate)
%matplotlib
import matplotlib.colors as colors
fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
#a=plt.plot(xf, np.abs(yf0))
#a=plt.plot(xf, np.abs(yf1))
a=ax[0].plot(xf, np.abs(yf0), '+', color='b')
a=ax[1].plot(xf, np.abs(yf1), '+', color='k')
a=ax[2].plot(xf, np.abs(yf2), '+', color='r')
#[ax[i].set_xlim(0,1) for i in [0,1,2]]

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
stim_spikes = []
for i in range(len(stim_obs.segments[0].spiketrains)):
    stim_spikes += [len(stim_obs.segments[0].spiketrains[i])]
    
exc_spikes = []
for i in range(len(exc_obs.segments[0].spiketrains)):
    exc_spikes += [len(exc_obs.segments[0].spiketrains[i])]
    
inh_spikes = []
for i in range(len(inh_obs.segments[0].spiketrains)):
    inh_spikes += [len(inh_obs.segments[0].spiketrains[i])]

fig, ax = plt.subplots(1,3,figsize=(11,5))
ax = ax.flatten()
p=ax[0].hist(stim_spikes)
p=ax[1].hist(exc_spikes)
p=ax[2].hist(inh_spikes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def connectionsGenerator(list_pre_idx, # pre
                         list_post_idx, # post
                         N_pop,
                         network_size,
                         weight,
                         min_delay,
                         max_delay,
                         number, # average number of presyn neurons per postsyn neuron
                         waveVelocity,
                         delay_ratio,
                         scale,
                         d_0,
                         expPar,
                         space,
                         conn_type,
                         pruning,
                         structure = "close",
                         do_autapse=False,
                         debug=False,
                         check_plots=True):

    # indices and SPACE
    # pre over lines
    pre_indices = np.arange(N_pop)[:, None] * np.ones((1, N_pop))
    # post over columns
    post_indices = np.arange(N_pop)[None, :] * np.ones((N_pop, 1))

    from pyNN.space import Line

    X = np.linspace(0,1, N_pop)
    DX = X[:, None] - X[None, :]

    if structure == 'close':
        DX = np.mod(DX + .5, 1) - .5

    d = np.sqrt(DX**2)

    prob = np.exp(- 1 / scale * np.abs(d/expPar) ** scale)
    #prob = scale * np.exp(- np.abs(d**2/2*width**2))
    #if debug: print(f'{do_autapse=}')
    if not do_autapse: prob[d==0.] = 0.
    prob /= prob.sum(axis=0)[None, :]                    
            
    # SYNAPSES
    # here, we want to control the amount of synapses reaching a neuron
    if debug: print(42*'-')
    if debug: print(N_pop, N_pop, 'shape of connectivity matrix=', prob.shape)
    # if debug: print(f'{prob.sum(axis=0).shape}')
    prob *= number 
    if debug: print(f'average prob per synapse {prob.mean():.5f}')
    if debug: print(f'max probability on synapses {prob.max():.5f}')
    # TODO : do something when the max is above 1
    if debug: print(42*'-')
    mask = prob > np.random.uniform(0, 1, size=prob.shape)
    if debug: print(f'number of synapses {mask.sum()}')
    if debug: print(f'average prob of synapse {mask.mean():.5f}')
    if debug: print(f'predicted average number of synapses per post neuron {prob.mean()*N_pop:.5f}')
    if debug: print(f'actual number of neurons per post neuron {mask.mean()*N_pop:.3f}')
        
    # WEIGHTS
    #weight_matrix = np.random.uniform(0.5*weight, 1.0*weight, size=mask.shape)
    weight_matrix = prob / prob.max() * weight
    weights = np.where(mask, weight_matrix, np.nan)
    if debug: print(42*'-')
    if debug: print(f'cumulative weight value {np.nansum(weights):.5f} [uS]')
        
    # Actual Projections width in a.u.
    d_ = d#*network_size*d_0
    if debug: 
        print(42*'-')
        print(f'max projections distance  {np.max(d_[weights>0]):.2f} [a.u.]')
        print(f'90th percentile projections distance {np.quantile(d_[weights>0], 0.9):.2f} [a.u.]')
        print(f'75th percentile projections distance {np.quantile(d_[weights>0], 0.75):.2f} [a.u.]')
        print(f'50th percentile projections distance {np.quantile(d_[weights>0], 0.50):.2f} [a.u.]')


    # DELAYS
    delays = min_delay + d*d_0/waveVelocity*delay_ratio
    delays = np.maximum(delays, min_delay)
    delays = np.minimum(delays, max_delay-min_delay) # 
    if debug: print(42*'-')
    if debug: print(f'mean delay {np.nanmean(delays):.5f} [ms]')   
    if debug: print(f'std delay {np.nanstd(delays):.5f} [ms]')       
    if debug: print(42*'-')
                                     
    # PUTTING ALL TOGETHER
    W = np.vstack((pre_indices.ravel()[mask.ravel()],
                   post_indices.ravel()[mask.ravel()],
                   weights.ravel()[mask.ravel()],
                   delays.ravel()[mask.ravel()]))
            



    return conn_list

In [6]:
min_delay = 1
max_delay = min_delay*144
d0 = 0.04
v = 0.5
distance = []
delay = []
weight_exc = 0.08
weight_inh = 0.08
conn_list_exc = []
conn_list_inh = []
import numpy as np
for i in range(1500):
    for j in range(1500):
        p = np.random.uniform(0,1)
        if p <= 0.02:
            distance_ = np.linalg.norm(i-j)
            delay_ = min_delay + distance_ * d0 / v
            #delay_ = np.maximum(delay_, min_delay)
            #delay_ = np.minimum(delay_, max_delay-min_delay) # 
            conn_list_exc += [[i, j, weight_exc, delay_]]
            conn_list_inh += [[i, j, weight_inh, delay_]]


import matplotlib.pyplot as plt
%matplotlib
fig, ax = plt.subplots(1,2)
ax = ax.flatten()
ax[1].plot(np.asarray(conn_list_exc).T[0], np.asarray(conn_list_exc).T[1], '+')
c=ax[0].hist(np.asarray(conn_list_exc).T[0])
print('synapses', len(conn_list_exc))

Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

synapses 45302


In [7]:
plt.figure()
plt.hist(np.asarray(conn_list_exc).T[3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([8709., 7606., 6711., 5783., 4966., 4152., 3246., 2253., 1360.,
         516.]),
 array([  1.   ,  12.928,  24.856,  36.784,  48.712,  60.64 ,  72.568,
         84.496,  96.424, 108.352, 120.28 ]),
 <a list of 10 Patch objects>)

In [8]:
%matplotlib
plt.hist(np.random.normal(loc=0.02, scale=0.002, size=1000))


Using matplotlib backend: module://ipympl.backend_nbagg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([  5.,  22.,  68., 169., 240., 234., 168.,  73.,  16.,   5.]),
 array([0.01361212, 0.01490456, 0.016197  , 0.01748944, 0.01878189,
        0.02007433, 0.02136677, 0.02265921, 0.02395165, 0.02524409,
        0.02653653]),
 <a list of 10 Patch objects>)

In [9]:
from pyNN.parameters import LazyArray
random_map = LazyArray(RandomDistribution('uniform', (0, 1), rng=None),
                       100)
connection_map = random_map < 0.02